In [1]:
import pandas as pd
from pathlib import Path
import numpy as np
import re
import h5py

In [2]:
with h5py.File('/data/IDLab/aar_foundation_models/processed_data/horse_1_ds_0.hdf5', 'r') as f:
    print(f['label'][:])

[b'unknown' b'unknown' b'unknown' ... b'unknown' b'unknown' b'unknown']
